## Thuật toán trích xuất k mốc nội suy từ giá trị x' 

In [1]:
import numpy as np
import pandas as pd
import math

def load_data_robust(filepath):
    """
    Hàm đọc dữ liệu dạng cột từ file txt một cách mạnh mẽ.
    - Bỏ qua các dòng trống.
    - Bỏ qua các dòng không phải là số (ví dụ: tiêu đề).
    - Tự động xử lý cả dấu phẩy (,) và dấu chấm (.).
    """
    data = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                # 1. Bỏ các khoảng trắng thừa ở đầu/cuối
                line = line.strip()
                
                # 2. Bỏ qua nếu là dòng trống
                if not line:
                    continue
                    
                # 3. Thay thế dấu phẩy bằng dấu chấm
                line = line.replace(',', '.')
                
                # 4. Thử chuyển đổi sang số, nếu thất bại (là chữ) thì bỏ qua
                try:
                    value = float(line)
                    data.append(value)
                except ValueError:
                    # Bỏ qua dòng này (ví dụ: đây là dòng tiêu đề)
                    pass
                    
    except FileNotFoundError:
        raise IOError(f"LỖI: Không tìm thấy file '{filepath}'.")
    except Exception as e:
        raise IOError(f"LỖI: Không thể đọc file '{filepath}'. Lỗi: {e}")
        
    return np.array(data)

# --- Chương trình chính cho Ô nạp dữ liệu ---
try:
    # 1. Nạp dữ liệu bằng hàm mới
    x_coords = load_data_robust('FileX.txt')
    y_coords = load_data_robust('FileY.txt')

    # 2. Kiểm tra dữ liệu (các bước kiểm tra giữ nguyên)
    if len(x_coords) == 0 or len(y_coords) == 0:
        raise ValueError("LỖI: Một trong hai file không chứa dữ liệu số hợp lệ.")
        
    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
        
    # Bạn có thể bỏ comment dòng kiểm tra "cách đều" nếu phương pháp của bạn yêu cầu
    # diffs = np.diff(x_coords)
    # if not np.allclose(diffs, diffs[0], rtol=1e-08, atol=1e-10): 
    #     raise ValueError("LỖI: Các mốc nội suy x phải cách đều nhau.")

    # 3. In kết quả
    print("Dữ liệu đã được đọc thành công!")
    print(f"Tổng số {len(x_coords)} điểm đã được nạp.")
    print("-" * 50)
    print("Các mốc x:", x_coords.tolist())
    print("Các giá trị y:", y_coords.tolist())
    
except (IOError, ValueError) as e:
    print(str(e))

Dữ liệu đã được đọc thành công!
Tổng số 201 điểm đã được nạp.
--------------------------------------------------
Các mốc x: [1.0, 1.118, 1.236, 1.354, 1.472, 1.59, 1.708, 1.826, 1.944, 2.062, 2.18, 2.298, 2.416, 2.534, 2.652, 2.77, 2.888, 3.006, 3.124, 3.242, 3.36, 3.478, 3.596, 3.714, 3.832, 3.95, 4.068, 4.186, 4.304, 4.422, 4.54, 4.658, 4.776, 4.894, 5.012, 5.13, 5.248, 5.366, 5.484, 5.602, 5.72, 5.838, 5.956, 6.074, 6.192, 6.31, 6.428, 6.546, 6.664, 6.782, 6.9, 7.018, 7.136, 7.254, 7.372, 7.49, 7.608, 7.726, 7.844, 7.962, 8.08, 8.198, 8.316, 8.434, 8.552, 8.67, 8.788, 8.906, 9.024, 9.142, 9.26, 9.378, 9.496, 9.614, 9.732, 9.85, 9.968, 10.086, 10.204, 10.322, 10.44, 10.558, 10.676, 10.794, 10.912, 11.03, 11.148, 11.266, 11.384, 11.502, 11.62, 11.738, 11.856, 11.974, 12.092, 12.21, 12.328, 12.446, 12.564, 12.682, 12.8, 12.918, 13.036, 13.154, 13.272, 13.39, 13.508, 13.626, 13.744, 13.862, 13.98, 14.098, 14.216, 14.334, 14.452, 14.57, 14.688, 14.806, 14.924, 15.042, 15.16, 15.278, 15.3

In [2]:
def extract_k_points(x_data, y_data, x_prime, k):
    """
    Trích xuất k điểm (x, y) gần nhất xung quanh x_prime.
    """
    N = len(x_data)
    
    # 1. Kiểm tra k hợp lệ
    if k > N:
        print(f"Cảnh báo: Yêu cầu k={k} điểm, nhưng chỉ có {N} điểm. Sẽ sử dụng tất cả {N} điểm.")
        return x_data, y_data
    
    # 2. Tìm chỉ số của mốc x gần nhất (bên trái)
    #    np.searchsorted tìm chỉ số để chèn x_prime vào mà vẫn giữ thứ tự
    start_index = np.searchsorted(x_data, x_prime) - 1
    
    # 3. Tính toán vị trí bắt đầu lý tưởng để cắt
    #    Chúng ta muốn 'start_index' nằm ở khoảng giữa của cửa sổ k điểm
    ideal_start = start_index - (k // 2) + 1
    
    # 4. Điều chỉnh vị trí bắt đầu để xử lý các vùng biên
    
    # Không thể bắt đầu trước chỉ số 0
    final_start = max(0, ideal_start)
    
    # Không thể bắt đầu quá muộn để rồi bị thiếu điểm
    # Vị trí bắt đầu cuối cùng có thể là N - k
    final_start = min(final_start, N - k)
            
    # 5. Cắt mảng để lấy đúng k điểm
    k_x_data = x_data[final_start : final_start + k]
    k_y_data = y_data[final_start : final_start + k]
    
    return k_x_data, k_y_data, final_start

print("Hàm 'extract_k_points' đã được định nghĩa.")

Hàm 'extract_k_points' đã được định nghĩa.


In [3]:
# --- Thiết lập đầu vào ---
x_prime_target = 1.9  # Điểm x' bạn muốn tính
k_points = 9          # Số lượng mốc nội suy bạn muốn sử dụng (ví dụ: 9)

# --- Thực thi trích xuất ---
try:
    k_x, k_y, start_idx = extract_k_points(x_coords, y_coords, x_prime_target, k_points)
    
    print(f"--- Trích xuất {k_points} điểm xung quanh x' = {x_prime_target} ---")
    
    # In ra kết quả (DataFrame)
    df_extracted = pd.DataFrame({
        'Vị trí (chỉ số gốc)': range(start_idx, start_idx + k_points),
        'x_trích_xuất': k_x,
        'y_trích_xuất': k_y
    })
    
    display(df_extracted.style.format("{:g}"))
    
    print("\nCác mốc này đã sẵn sàng để được sử dụng cho các hàm nội suy (Gauss, Newton, v.v.)")
    
    # >>> CÁC DÒNG MỚI ĐƯỢC THÊM VÀO Ở ĐÂY <<<
    print("\n" + "="*50)
    print(">>> Dữ liệu để sao chép vào FileX.txt (theo chiều ngang):")
    print(" ".join(map(str, k_x)))
    
    print("\n>>> Dữ liệu để sao chép vào FileY.txt (theo chiều ngang):")
    print(" ".join(map(str, k_y)))
    print("="*50)
    
except NameError:
    print("LỖI: Biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy Ô 1 trước.")

--- Trích xuất 9 điểm xung quanh x' = 1.9 ---


,Vị trí (chỉ số gốc),x_trích_xuất,y_trích_xuất
0,4,1.472,2.7958
1,5,1.59,3.1802
2,6,1.708,3.5347
3,7,1.826,3.855
4,8,1.944,4.1369
5,9,2.062,4.3769
6,10,2.18,4.572
7,11,2.298,4.7196
8,12,2.416,4.8182



Các mốc này đã sẵn sàng để được sử dụng cho các hàm nội suy (Gauss, Newton, v.v.)

>>> Dữ liệu để sao chép vào FileX.txt (theo chiều ngang):
1.472 1.59 1.708 1.826 1.944 2.062 2.18 2.298 2.416

>>> Dữ liệu để sao chép vào FileY.txt (theo chiều ngang):
2.7958 3.1802 3.5347 3.855 4.1369 4.3769 4.572 4.7196 4.8182
